# Futures trend following with custom args

In [1]:
# Import libraries
import xarray as xr
import xarray.ufuncs as xruf
import numpy as np
import pandas as pd

import qnt.output as qnout
import qnt.ta as qnta
import qnt.data    as qndata
import qnt.stats   as qnstats
import qnt.graph   as qngraph
import datetime    as dt
import plotly.graph_objs as go
import xarray.ufuncs as xruf
import time

#### Load futures data
Quantnet provides data for 75 global derivatives. 
The underlying assets are currencies, cross-rates, indices, bonds, energy and metals from the world's futures exchanges.

Suppose we want to download the data for the last 20 years. One can use the following function:

In [2]:
fut_data = qndata.futures.load_data(tail = 20*365)
# The complete list
fut_data.asset

100% (33771540 of 33771540) |############| Elapsed Time: 0:00:00 Time:  0:00:00


<xarray.DataArray 'asset' (asset: 75)>
array(['F_AD', 'F_AE', 'F_AH', 'F_AX', 'F_BC', 'F_BG', 'F_BO', 'F_BP', 'F_C',
       'F_CA', 'F_CC', 'F_CD', 'F_CF', 'F_CL', 'F_CT', 'F_DM', 'F_DT', 'F_DX',
       'F_EB', 'F_EC', 'F_ED', 'F_ES', 'F_F', 'F_FB', 'F_FC', 'F_FP', 'F_FV',
       'F_FY', 'F_GC', 'F_GS', 'F_GX', 'F_HG', 'F_HO', 'F_JY', 'F_KC', 'F_LB',
       'F_LC', 'F_LN', 'F_LR', 'F_LX', 'F_MD', 'F_MP', 'F_ND', 'F_NG', 'F_NQ',
       'F_NR', 'F_NY', 'F_O', 'F_OJ', 'F_PA', 'F_PL', 'F_RB', 'F_RF', 'F_RP',
       'F_RR', 'F_RU', 'F_RY', 'F_S', 'F_SB', 'F_SF', 'F_SI', 'F_SM', 'F_SS',
       'F_SX', 'F_TR', 'F_TU', 'F_TY', 'F_UB', 'F_US', 'F_UZ', 'F_VX', 'F_W',
       'F_XX', 'F_YM', 'F_ZQ'], dtype=object)
Coordinates:
  * asset    (asset) object 'F_AD' 'F_AE' 'F_AH' 'F_AX' ... 'F_XX' 'F_YM' 'F_ZQ'

In [3]:
# we can see historical data on a chart
trend_fig = [
    go.Scatter(
        x = fut_data.sel(asset = 'F_DX').sel(field = 'close').to_pandas().index,
        y = fut_data.sel(asset = 'F_DX').sel(field = 'close'),
        line = dict(width=1,color='black'))]

# draw chart
fig = go.Figure(data = trend_fig)
fig.update_yaxes(fixedrange=False) # unlock vertical scrolling
fig.show()

## Weights allocation

This function calculates positions using wma and roc as trend indicators.

In [4]:
def calc_positions(futures, ma_periods, roc_periods, sideways_threshold):
    """ Calculates positions for given data(futures) and parameters """
    close = futures.sel(field='close')
    
    # calculate MA 
    ma = qnta.lwma(close, ma_periods)
    # calcuate ROC
    roc = qnta.roc(ma, roc_periods)

    # positive trend direction
    positive_trend = roc > sideways_threshold
    # negtive trend direction
    negative_trend = roc < -sideways_threshold 
    # sideways
    sideways_trend = abs(roc) <= sideways_threshold
    
    # We suppose that a sideways trend after a positive trend is also positive
    side_positive_trend = positive_trend.where(sideways_trend == False).ffill('time').fillna(False)
    # and a sideways trend after a negative trend is also negative
    side_negative_trend = negative_trend.where(sideways_trend == False).ffill('time').fillna(False)

    # define signals
    buy_signal = positive_trend
    buy_stop_signal = side_negative_trend

    sell_signal = negative_trend
    sell_stop_signal = side_positive_trend

    # calc positions 
    position = close.copy(True)
    position[:] = np.nan
    position = xr.where(buy_signal, 1, position)
    position = xr.where(sell_signal, -1, position)
    position = xr.where(xruf.logical_and(buy_stop_signal, position.ffill('time') > 0), 0, position)
    position = xr.where(xruf.logical_and(sell_stop_signal, position.ffill('time') < 0), 0, position)
    position = position.ffill('time').fillna(0)

    return position

Select asset and adjust parameters:

In [5]:
asset = 'F_DX' ###

sdat = fut_data.sel(asset=asset).dropna('time','any')
sout = calc_positions(sdat, 40, 6, 1)
sout = xr.concat([sout], pd.Index([asset], name='asset'))

ssta = qnstats.calc_stat(fut_data, sout)

display(ssta.to_pandas().tail())

performance = ssta.to_pandas()["equity"]
qngraph.make_plot_filled(performance.index, performance, name="PnL (Equity)", type="log")

field,equity,relative_return,volatility,underwater,max_drawdown,sharpe_ratio,mean_return,bias,instruments,avg_turnover,avg_holding_time
time,,,,,,,,,,,
2021-01-19,1.654408,-0.003062,0.077413,-0.125410,-0.209087,0.320252,0.024792,1.0,1.0,0.019766,55.814286
2021-01-20,1.654279,-0.000078,0.077405,-0.125478,-0.209087,0.320170,0.024783,1.0,1.0,0.019763,55.814286
2021-01-21,1.648031,-0.003777,0.077402,-0.128781,-0.209087,0.317682,0.024589,1.0,1.0,0.019760,55.814286
2021-01-22,1.649584,0.000943,0.077395,-0.127960,-0.209087,0.318256,0.024631,1.0,1.0,0.019757,55.814286
2021-01-25,1.652528,0.001784,0.077389,-0.126404,-0.209087,0.319368,0.024715,1.0,1.0,0.019753,69.760563


This function calculate positions for multiple instruments with different parameters.

In [6]:
def calc_output_all(data, params):
    positions = data.sel(field='close').copy(True)
    positions[:] = np.nan
    for futures_name in params.keys(): 
        p = params[futures_name]
        futures_data = data.sel(asset=futures_name).dropna('time','any')
        p = calc_positions(futures_data, p['ma_periods'], p['roc_periods'], p['sideways_threshold'])
        positions.loc[{'asset':futures_name, 'time':p.time}] = p
    
    return positions

In [7]:
# say we select futures and their parameters for technical algorithm
params = {
    'F_NY': {
        'ma_periods': 200, 
        'roc_periods': 5, 
        'sideways_threshold': 2,
    },
    'F_GX': {
        'ma_periods': 200, 
        'roc_periods': 20, 
        'sideways_threshold': 2
    },
    'F_DX': {
        'ma_periods': 40, 
        'roc_periods': 6, 
        'sideways_threshold': 1
    },
}

futures_list = list(params.keys())

# form the output
output = calc_output_all(fut_data.sel(asset = futures_list), params)

# check the output
qnout.check(output, fut_data)

# write the result
qnout.write(output)

# show statistics
stat = qnstats.calc_stat(fut_data, output.sel(time=slice('2006-01-01', None)))
display(stat.to_pandas().tail())

# show plot with profit and losses:
performance = stat.to_pandas()["equity"]
qngraph.make_plot_filled(performance.index, performance, name="PnL (Equity)", type="log")

Check missed dates...
Ok.
Check sharpe ratio.


ERROR! The sharpe ratio is too low. 0.5713306606684113 < 1


Check correlation.

Ok. This strategy does not correlate with other strategies.
Write output: /root/fractions.nc.gz


field,equity,relative_return,volatility,underwater,max_drawdown,sharpe_ratio,mean_return,bias,instruments,avg_turnover,avg_holding_time
time,,,,,,,,,,,
2021-01-19,1.990816,-0.001743,0.081211,-0.007951,-0.145303,0.559547,0.045442,1.0,3.0,0.043692,38.632288
2021-01-20,1.989190,-0.000817,0.081201,-0.008762,-0.145303,0.558791,0.045375,1.0,3.0,0.043852,38.582342
2021-01-21,1.978467,-0.005391,0.081203,-0.014105,-0.145303,0.554143,0.044998,1.0,3.0,0.043842,38.582342
2021-01-22,1.981404,0.001485,0.081193,-0.012641,-0.145303,0.555295,0.045086,1.0,3.0,0.043833,38.582342
2021-01-25,1.993780,0.006246,0.081198,-0.006474,-0.145303,0.560289,0.045494,1.0,3.0,0.043823,38.386139


# Multi-pass implementation

Now, let's use multi-pass approach to verify the strategy. It is much slower but it is the best way to properly test it and to avoid looking-forward.

In [8]:
%%javascript
IPython.OutputArea.prototype._should_scroll = function(lines) { return false; }
// disable widget scrolling

<IPython.core.display.Javascript object>

In [9]:
# In your final submission you can remove/deactivate all the other cells to reduce the checking time.
# The checking system will run this book multiple times for every trading day within the in-sample period.
# Every pass the available data will be isolated till the current day.
# qnt.backtester is optimized to work with the checking system.
# The checking system will override test_period=1 to make your strategy to produce weights for 1 day per pass.

import xarray as xr
import numpy as np

import qnt.ta as qnta
import qnt.backtester as qnbt
import qnt.data as qndata
import qnt.xr_talib as xrtl
import xarray.ufuncs as xruf
import qnt.ta as qnta


def load_data(period):
    return qndata.futures_load_data(tail=period)


def calc_positions(futures, ma_periods, roc_periods, sideways_threshold):
    """ Calculates positions for given data(futures) and parameters """
    close = futures.sel(field='close')
    
    # calculate MA 
    ma = qnta.lwma(close, ma_periods)
    # calcuate ROC
    roc = qnta.roc(ma, roc_periods)

    # positive trend direction
    positive_trend = roc > sideways_threshold
    # negtive trend direction
    negative_trend = roc < -sideways_threshold 
    # sideways
    sideways_trend = abs(roc) <= sideways_threshold
    
    # We suppose that a sideways trend after a positive trend is also positive
    side_positive_trend = positive_trend.where(sideways_trend == False).ffill('time').fillna(False)
    # and a sideways trend after a negative trend is also negative
    side_negative_trend = negative_trend.where(sideways_trend == False).ffill('time').fillna(False)

    # define signals
    buy_signal = positive_trend
    buy_stop_signal = side_negative_trend

    sell_signal = negative_trend
    sell_stop_signal = side_positive_trend

    # calc positions 
    position = close.copy(True)
    position[:] = np.nan
    position = xr.where(buy_signal, 1, position)
    position = xr.where(sell_signal, -1, position)
    position = xr.where(xruf.logical_and(buy_stop_signal, position.ffill('time') > 0), 0, position)
    position = xr.where(xruf.logical_and(sell_stop_signal, position.ffill('time') < 0), 0, position)
    position = position.ffill('time').fillna(0)

    return position


def calc_output_all(data, params):
    positions = data.sel(field='close').copy(True)
    positions[:] = np.nan
    for futures_name in params.keys(): 
        p = params[futures_name]
        futures_data = data.sel(asset=futures_name).dropna('time','any')
        p = calc_positions(futures_data, p['ma_periods'], p['roc_periods'], p['sideways_threshold'])
        positions.loc[{'asset':futures_name, 'time':p.time}] = p
    
    return positions

# say we select futures and their parameters for technical algorithm
params = {
    'F_NY': {
        'ma_periods': 200, 
        'roc_periods': 5, 
        'sideways_threshold': 2,
    },
    'F_GX': {
        'ma_periods': 200, 
        'roc_periods': 20, 
        'sideways_threshold': 2
    },
    'F_DX': {
        'ma_periods': 40, 
        'roc_periods': 6, 
        'sideways_threshold': 1
    },
}
futures_list = list(params.keys())


def strategy(data):
    output = calc_output_all(data.sel(asset = futures_list), params)
    return output.isel(time=-1)


weights = qnbt.backtest(
    competition_type="futures",
    load_data=load_data,
    lookback_period=5*365,
    start_date='2006-01-01',
    strategy=strategy
)

Run last pass...
Load data...
Run pass...
Ok.
---
Run first pass...
Load data...
Run pass...
Ok.
---
Load full data...
---
Run iterations...



100% (3894 of 3894) |####################| Elapsed Time: 0:02:47 Time:  0:02:47


Merge outputs...
Load data for cleanup and analysis...
Check missed dates...
Ok.
Normalization...
Done.
Write output: /root/fractions.nc.gz
---
Analyze results...
Check...
Check missed dates...
Ok.
Check sharpe ratio.


ERROR! The sharpe ratio is too low. 0.5037131652680984 < 1


Check correlation.

Ok. This strategy does not correlate with other strategies.
---
Calc global stats...
---
Calc stats per asset...
Build plots...
---
Select the asset (or leave blank to display the overall stats):


interactive(children=(Combobox(value='', description='asset', options=('', 'F_DX', 'F_GX', 'F_NY')), Output())…

### 